In [1]:
import advent_of_code_2025
points = advent_of_code_2025.get_csv_muliline_py(9,True,1)

In [2]:
points

[[7, 1], [11, 1], [11, 7], [9, 7], [9, 5], [2, 5], [2, 3], [7, 3]]

In [3]:
def area(p,q):
    return (abs(p[0] - q[0]) + 1) * (abs(p[1] - q[1]) + 1)
max(area(p,q) for i,p in enumerate(points) for q in points[:i])

50

In [4]:
permit_space = {(x,y):False for x in [x for x,y in points] for y in [y for x,y in points]}

In [5]:
for (a,b),(c,d) in zip(points,points[1:]+[points[0]]):
    if a == c:
        if b <= d:
            for y in range(b,d+1):
                if (a,y) in permit_space:
                    permit_space[a,y] = True
        else:
            for y in range(d,b+1):
                if (a,y) in permit_space:
                    permit_space[a,y] = True
    else: #assume b == d
        if a <= c:
            for x in range(a,c+1):
                if (x,b) in permit_space:
                    permit_space[x,b] = True
        else:
            for x in range(c,a+1):
                if (x,b) in permit_space:
                    permit_space[x,b] = True


In [6]:
def print_permit(permit_space):
    n = len(set(x for x,y in permit_space))
    m = len(set(y for x,y in permit_space))
    grid = [['.' for _ in range(m)] for _ in range(n)]
    s = [(x,y,b) for (x,y),b in permit_space.items()]
    s.sort()
    for i,(_,_,b) in enumerate(s):
        grid[i//n][i%m] = 'X' if b else '.'
    return grid

In [7]:
print_permit(permit_space)

[['.', 'X', 'X', '.'],
 ['X', 'X', 'X', '.'],
 ['X', '.', 'X', 'X'],
 ['X', 'X', 'X', 'X']]

In [8]:
import sys
sys.setrecursionlimit(10000)
def fill_interior(permit_space):
    "method is exclude anything connected to the boundary"
    def fill(x,y,m):
        m[x][y] = ' '
        for i,j in [(x-1,y),(x+1,y),(x,y-1),(x,y+1)]:
            if 0 <= i < len(m) and 0 <= j < len(m[0]) and m[i][j] == '.':
                m = fill(i,j,m)
        return m
    mini_map = print_permit(permit_space)
    n = len(mini_map)
    m = len(mini_map[0])
    for x,y in ([(0,y) for y in range(m)]
        + [(len(mini_map[0])-1,y) for y in range(m)]
        + [(x,0) for x in range(n)]
        + [(x,len(mini_map)-1) for x in range(n)]):
        if mini_map[x][y] == '.':
            mini_map = fill(x,y,mini_map)
    s = [(x,y,b) for (x,y),b in permit_space.items()]
    s.sort()
    for i,(x,y,_) in enumerate(s):
        permit_space[(x,y)] = mini_map[i//n][i%m] != ' '
    return permit_space

In [9]:
permit_space_filled = fill_interior(permit_space)

In [10]:
x_vals = sorted(list(set(x for x,y in permit_space)))
y_vals = sorted(list(set(y for x,y in permit_space)))

def boundary_inside(p,q,permit_space_filled):
    a,b = min(p[0],q[0]),min(p[1],q[1])
    c,d = max(p[0],q[0]),max(p[1],q[1])
    x_bits = [x for x in x_vals if a <= x <= c]
    y_bits = [y for y in y_vals if b <= y <= d]
    return all(permit_space_filled[(x,y)] for x in x_bits for y in y_bits)

max(area(p,q) for i,p in enumerate(points) for q in points[:i] if boundary_inside(p,q,permit_space_filled))

24